## PySpark Random Sample

PySpark provides a `pyspark.sql.DataFrame.sample()`, `pyspark.sql.DataFrame.sampleBy()`, `RDD.sample()`, and `RDD.takeSample()` methods to get the random sampling subset from the large dataset

In [0]:
dbutils.library.restartPython() # Removes Python state, but some libraries might not work without calling this command.dbutils.restartPython()

#### Load libraries

In [0]:
from pyspark.sql import SparkSession, Row
from pyspark.sql.types import IntegerType, DateType, StringType, StructType, StructField, ArrayType, MapType, DoubleType
from pyspark.sql.functions import lit, col, expr, when, sum, avg, max, min, mean, count, udf, explode, concat_ws

#### Create Spark session

In [0]:
spark = SparkSession.builder.appName('PySpark Random Sample').getOrCreate()

#### Using sample

By using fraction between 0 to 1, it returns the approximate number of the fraction of the dataset. For example, 0.1 returns 10% of the rows. However, this does not guarantee it returns the exact 10% of the records.

In [0]:
df=spark.range(100)
print(df.sample(0.06).collect())

[Row(id=10), Row(id=51), Row(id=57), Row(id=66), Row(id=72), Row(id=90), Row(id=96)]

#### Using seed to reproduce the same Samples 

Every time you run a `sample()`` function it returns a different set of sampling records, however sometimes during the development and testing phase you may need to regenerate the same sample every time as you need to compare the results from your previous run.

In [0]:
print(df.sample(0.1,123).collect())
print(df.sample(0.1,123).collect())

[Row(id=10), Row(id=21), Row(id=34), Row(id=72), Row(id=84), Row(id=91)]
[Row(id=10), Row(id=21), Row(id=34), Row(id=72), Row(id=84), Row(id=91)]

#### Sample withReplacement (May contain duplicates)

Some times you may need to get a random sample with repeated values. By using the value `true`, results in repeated values.

In [0]:
print(df.sample(True,0.3,123).collect()) # with duplicates
print(df.sample(False,0.3,123).collect()) # without duplicates
print(df.sample(0.3,123).collect()) # without duplicates

[Row(id=0), Row(id=4), Row(id=7), Row(id=7), Row(id=9), Row(id=12), Row(id=12), Row(id=20), Row(id=22), Row(id=24), Row(id=28), Row(id=30), Row(id=30), Row(id=31), Row(id=32), Row(id=37), Row(id=37), Row(id=39), Row(id=48), Row(id=50), Row(id=54), Row(id=55), Row(id=66), Row(id=69), Row(id=78), Row(id=82), Row(id=83), Row(id=90), Row(id=91), Row(id=92), Row(id=94), Row(id=94), Row(id=97)]
[Row(id=0), Row(id=3), Row(id=6), Row(id=10), Row(id=12), Row(id=14), Row(id=16), Row(id=21), Row(id=32), Row(id=34), Row(id=35), Row(id=41), Row(id=42), Row(id=44), Row(id=46), Row(id=48), Row(id=49), Row(id=59), Row(id=64), Row(id=72), Row(id=77), Row(id=81), Row(id=83), Row(id=84), Row(id=85), Row(id=86), Row(id=91), Row(id=95), Row(id=97), Row(id=98)]
[Row(id=0), Row(id=3), Row(id=6), Row(id=10), Row(id=12), Row(id=14), Row(id=16), Row(id=21), Row(id=32), Row(id=34), Row(id=35), Row(id=41), Row(id=42), Row(id=44), Row(id=46), Row(id=48), Row(id=49), Row(id=59), Row(id=64), Row(id=72), Row(id=77), Row(id=81), Row(id=83), Row(id=84), Row(id=85), Row(id=86), Row(id=91), Row(id=95), Row(id=97), Row(id=98)]

#### Stratified sampling

You can get Stratified sampling in PySpark without replacement by using `sampleBy()` method. It returns a sampling fraction for each stratum. If a stratum is not specified, it takes zero as the default.

In [0]:
df2=df.select((df.id % 3).alias('key'))
print(df2.sampleBy('key', {0: 0.1, 1: 0.2},0).collect())

[Row(key=1), Row(key=0), Row(key=1), Row(key=0), Row(key=0), Row(key=1), Row(key=1), Row(key=1), Row(key=1), Row(key=1), Row(key=0), Row(key=1), Row(key=1), Row(key=0), Row(key=1), Row(key=1), Row(key=0), Row(key=1), Row(key=1)]

#### RDD Sample

PySpark RDD also provides `sample()` function to get a random sampling, it also has another signature `takeSample()` that returns an Array[].

In [0]:
rdd = spark.sparkContext.range(0,100)
print(rdd.sample(False,0.1,0).collect())
print(rdd.sample(True,0.3,123).collect())

[28, 43, 47, 51, 58, 67, 82]
[0, 10, 13, 13, 14, 16, 19, 29, 29, 30, 35, 53, 54, 57, 58, 60, 61, 64, 85, 93, 96]

In [0]:
#RDD takeSample() is an action you need to be careful using as it returns the selected sample records to driver memory.
#Returning too much data results in an out-of-memory error similar to collect().
print(rdd.takeSample(False,10,0))

[26, 63, 45, 68, 30, 1, 34, 89, 82, 79]

In [0]:
print(rdd.takeSample(True,30,123))

[92, 34, 47, 72, 84, 54, 18, 57, 38, 12, 16, 82, 56, 42, 53, 30, 10, 42, 6, 61, 29, 56, 55, 88, 18, 36, 79, 76, 29, 53]

#### The end of the notebook